In [1]:
import xarray as xr

In [5]:
ds = xr.open_zarr("https://mur-sst.s3.us-west-2.amazonaws.com/zarr", consolidated=True)
ds

,Array,Chunk
Bytes,15.19 TiB,245.78 MiB
Shape,"(6443, 17999, 36000)","(6443, 100, 100)"
Count,64801 Tasks,64800 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,15.19 TiB,245.78 MiB
Shape,"(6443, 17999, 36000)","(6443, 100, 100)"
Count,64801 Tasks,64800 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [6]:
lat_min = 26
lat_max = 43
lon_min = -130
lon_max = -110
ilat = (ds.lat >= lat_min) & (ds.lat <= lat_max)
ilon = (ds.lon >= lon_min) & (ds.lon <= lon_max)
sst = ds.analysed_sst.isel(lat=ilat, lon=ilon)
sst

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


,Array,Chunk
Bytes,81.70 GiB,245.78 MiB
Shape,"(6443, 1701, 2001)","(6443, 100, 100)"
Count,68959 Tasks,378 Chunks
Type,float32,numpy.ndarray


In [7]:
sst_mon = sst.where(sst != 265.382).resample(time='M').mean('time')
sst_mon

,Array,Chunk
Bytes,155.81 MiB,39.06 kiB
Shape,"(12, 1701, 2001)","(1, 100, 100)"
Count,590599 Tasks,4536 Chunks
Type,float32,numpy.ndarray


In [9]:
sst_mon = sst_mon.compute()
sst_mon.to_netcdf('/home/jovyan/shared/data/ecopro/KelpForest/MUR_mon_200206-202001.nc')